In [4]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # load an official detection model
results = model.predict(source="0", show=True, classes=[0], conf=0.8)  # Display preds. Accepts all YOLO predict arguments



1/1: 0... Success ✅ (inf frames of shape 1920x1080 at 30.00 FPS)


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 384x640 1 person, 60.2ms
0: 384x640 1 person, 39.5ms
0: 384x640 1 person, 50.5ms
0: 384x640 1 person, 50.7ms
0: 384x640 1 person, 53.1ms
0: 384x640 1 person, 43.9ms
0: 384x640 1 person, 53.1ms
0: 384x640 1 person, 54.4ms
0: 384x640 1 person, 37.3ms
0: 384x640 1 person, 38.4ms
0: 384x640 1 person, 30.9ms
0: 384x640 1 person, 41.0ms
0: 384x640 1 person, 56.2ms
0: 384x640 1 person, 54.1m

KeyboardInterrupt: 